In [ ]:
# Defining random seeds to enable reproducibility
from numpy.random import seed
seed(1)

import tensorflow as tf
tf.random.set_seed(1)

import random
random.seed(1)

import cv2
import pickle
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import make_axes_locatable
from tf_explain.core.grad_cam import GradCAM
from tensorflow.python.client import device_lib
from tensorflow.keras.models import load_model
from sklearn.model_selection import train_test_split


print(device_lib.list_local_devices())
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))


def load_dataset():
    """
    Loads the datasets encoded in .pkl files and returns its decoded form.

    Returns
    -------
    list
        A list of n-dimensional arrays representing the subjects samples that will be used to \\
        train the drunkenness classification model.
    ndarray
        A n-dimensional array representing the samples labels.
    """

    print("Loading Sober-Drunk Face Dataset, from Patras University")
    
    # Defining the sample and label sets filenames
    sets = [
        "Insert the x_imbalanced.pkl file path here",
        "Insert the y_imbalanced.pkl file path here"   
    ]
    
    # Defining an empty list for storing the decoded dataset
    loaded_datasets = []
 
    # Iterating over the dataset files
    for set_ in sets:
        # Opening the .pkl file in read mode
        with open(set_, 'rb') as file:
            # Appending the decoded dataset to the dataset list
            loaded_datasets.append(pickle.load(file))
    
    # Unpacking the dataset list into individual subsets
    x, y = loaded_datasets
    
    # Converting the label list to the n-dimensional array format
    y_arr= np.array(y)
    
    # Printing the dataset length for sanity check
    print("\nSamples total: {0}".format(len(x)))
    
    # Slicing the frame sequences of each subject for selecting the
    # frames sampled at each 5 Hz
    x = x[::5]
    y_arr = y_arr[::5]
    
    # Printing the dataset length after slicing for sanity check
    print("\nSamples total after slicing: {0}".format(len(x)))
    
    # Returning the samples set and its respective labels
    return x, y_arr


def min_max_norm(dataset):
    """
    Normalizes the keyframes according to the minimum-maximum norm, \\
    such that pixel values ranges from 0 to 1.

    Parameters
    ----------
    dataset : list
        A list of n-dimensional arrays representing the subjects keyframes.

    Returns
    -------
    ndarray
        A n-dimensional array representing keyframes with pixel values ranging from 0 to 1.
    """

    # Converting the dataset type from list to n-dimensional array
    dataset = np.asarray(dataset, dtype="int16")

    # Finding the keyframes minimum and maximum values
    x_min = dataset.min(axis=(1, 2), keepdims=True)
    x_max = dataset.max(axis=(1, 2), keepdims=True)

    # Applying the minimum-maximum norm to each keyframe
    norm_dataset = (dataset - x_min) / (x_max - x_min)

    # Printing the minimum and maximum values from a given sample for sanity check
    print("\nMinMax normalization")
    print("dataset shape: ", norm_dataset.shape)
    print("min: ", norm_dataset[0].min())
    print("max: ", norm_dataset[0].max())

    # Returning the normalized dataset
    return norm_dataset


def visualize_gradcam(cam, filename, operation):
    """
    Displays the class activation mapping for a given test set sample.

    Parameters
    ----------
    cam : ndarray
        A n-dimensional array representing the class activation map.
    filename : str
        A string indicating the filename to save the plot.
    operation : str
        A string indicating the operation result to be displayed.
    """

    # Creating a new figure
    plt.figure()

    # Getting the figure current axis
    ax = plt.gca()
    
    # Displaying the class activation mapping
    im = ax.imshow(cam, cmap='turbo', interpolation='lanczos')
    
    # Hidding the axis ticks
    plt.xticks([])
    plt.yticks([])
    
    # Defining a colorbar to interpret the class activations relevance
    divider = make_axes_locatable(ax)
    cax = divider.append_axes("right", size="5%", pad=0.05)
    plt.colorbar(im, cax=cax)
    
    # Saving the given sample class activation mapping
    plt.savefig("{0}_{1}.pdf".format(filename, operation), dpi=600, bbox_inches='tight', pad_inches=0.001)
    
    # Showing the given sample class activation mapping
    plt.show()


def gradcam_frame_comparison():
     """
    Performs the Gradient-weighted Class Activation Mapping visualization technique \\
    for the frame sequence of a subject.
    """
    
    # Defining the layer whose gradients will be used to highlight the predicted class 
    # discriminating features
    LAYER_NAME = 'block5_conv3'
    
    # Loading the final drunkenness classification model
    model = load_model('sober-drunk-cc_vgg16_final-model-ft_data_aug_tts_70-30_L2_405e.h5')
    
    # Printing the final model summary
    model.summary()
    
    # Loading the Sober-Drunk Dataset samples
    x, y = load_dataset()

    # Selecting samples from the subjects 37, 38, 39, 40 and 41
    x, y = x[-200:], y[-200:]

    # Applying the min-max normalization
    x = min_max_norm(x)

    # Defining the filenames of each sample that will be analized
    subjects = ['37_vassilisA_1_f_M_54_90',
                '37_vassilisA_2_f_M_54_90',
                '37_vassilisA_3_f_M_54_90',
                '37_vassilisA_4_f_M_54_90',
                '38_christos_1_f_M_33_76',
                '38_christos_2_f_M_33_76',
                '38_christos_3_f_M_33_76',
                '38_christos_4_f_M_33_76',
                '39_anna_1_f_F_46_59',
                '39_anna_2_f_F_46_59',
                '39_anna_3_f_F_46_59',
                '39_anna_4_f_F_46_59',
                '40_spiliop_1_f_M_43_74',
                '40_spiliop_2_f_M_43_74',
                '40_spiliop_3_f_M_43_74',
                '40_spiliop_4_f_M_43_74',
                '41_kalp_1_f_M_62_95',
                '41_kalp_2_f_M_62_95',
                '41_kalp_3_f_M_62_95',
                '41_kalp_4_f_M_62_95']

    # Defining a list of indexes for the frame sequence
    frame_idx = 0
    frame_list = range(0,50,5)
    
    # Defining the index of subject 38 filename
    subject_idx = 4

    # Iterating over the subject 38 samples
    for sample in x[40:80]:
        # Reshaping the input image format
        input_img = np.expand_dims(sample, axis=0)        
        data = (input_img, None)

        print("Subject {0}, frame {1}".format(subjects[subject_idx], frame_list[frame_idx]))

        # Instantiating the Grad-CAM explainer object
        explainer = GradCAM()

        # Running the Grad-CAM explainer
        cam = explainer.explain(data, model, class_index=-1, colormap=cv2.COLORMAP_TURBO, image_weight=0.0)

        # Displaying the raw class activation map
        visualize_gradcam(cam, "{0}_frame-{1}".format(subjects[subject_idx], frame_list[frame_idx]), 'turbo', 'cam')

        # Normalizing the input image
        input_img = sample 
        input_img = cv2.normalize(input_img,  np.zeros((128, 160)), 0, 255, cv2.NORM_MINMAX, dtype=cv2.CV_8U)

        # Defining the transparency weights
        alpha = 0.5
        beta = 1 - alpha
        gamma = 0
        
        # Superimposing the class activation map over the input image
        overlayed_img = cv2.addWeighted(input_img, alpha, cam, beta, gamma)

        # Displaying the overlaid class activation mapping
        visualize_gradcam(overlayed_img, "{0}_frame-{1}".format(subjects[subject_idx], frame_list[frame_idx]), 'turbo', 'grad-cam_blending')

        # Increasing the frame index 
        frame_idx += 1

        # Checking if the frame index is equal or higher than the number
        # of frames in the frame sequence
        if frame_idx >= 10:
            # Resetting the frame index for the next sequence
            frame_idx = 0
            # Increasing the subject index
            subject_idx += 1


def gradcam_subject_analysis():
    """
    Performs the Gradient-weighted Class Activation Mapping visualization technique \\
    for the first frame of each subject.
    """
    
    # Defining the layer whose gradients will be used to highlight the predicted class 
    # discriminating features
    LAYER_NAME = 'block5_conv3'
    
    # Loading the final drunkenness classification model
    model = load_model('sober-drunk-cc_vgg16_final-model-ft_data_aug_tts_70-30_L2_405e.h5')
    
    # Printing the final model summary
    model.summary()
    
    # Loading the Sober-Drunk Dataset samples
    x, y = load_dataset()

    # Selecting samples from the subjects 37, 38, 39, 40 and 41
    x, y = x[-200:], y[-200:]

    # Applying the min-max normalization
    x = min_max_norm(x)   
    
    # Defining the filenames of each sample that will be analized
    subjects = ['37_vassilisA_1_f_M_54_90',
                '37_vassilisA_2_f_M_54_90',
                '37_vassilisA_3_f_M_54_90',
                '37_vassilisA_4_f_M_54_90',
                '38_christos_1_f_M_33_76',
                '38_christos_2_f_M_33_76',
                '38_christos_3_f_M_33_76',
                '38_christos_4_f_M_33_76',
                '39_anna_1_f_F_46_59',
                '39_anna_2_f_F_46_59',
                '39_anna_3_f_F_46_59',
                '39_anna_4_f_F_46_59',
                '40_spiliop_1_f_M_43_74',
                '40_spiliop_2_f_M_43_74',
                '40_spiliop_3_f_M_43_74',
                '40_spiliop_4_f_M_43_74',
                '41_kalp_1_f_M_62_95',
                '41_kalp_2_f_M_62_95',
                '41_kalp_3_f_M_62_95',
                '41_kalp_4_f_M_62_95']
    
    # Selecting the 1st frame of each frame sequence
    subject_samples = x[::10]
    
    # Iterating over the subject samples
    for i in range(len(subject_samples)):
        # Reshaping the input image format
        input_img = np.expand_dims(subject_samples[i], axis=0)        
        data = (input_img, None)
        
        print("Subject {0}".format(subjects[i]))
        
        # Instantiating the Grad-CAM explainer object
        explainer = GradCAM()
        
        # Running the Grad-CAM explainer
        cam = explainer.explain(data, model, class_index=-1, colormap=cv2.COLORMAP_TURBO, image_weight=0.0)
        
        # Displaying the raw class activation map
        visualize_gradcam(cam, subjects[i], 'turbo', 'cam')
        
        # Normalizing the input image
        input_img = subject_samples[i] 
        input_img = cv2.normalize(input_img,  np.zeros((128, 160)), 0, 255, cv2.NORM_MINMAX, dtype=cv2.CV_8U)
        
        # Defining the transparency weights
        alpha = 0.5
        beta = 1 - alpha
        gamma = 0
        
        # Superimposing the class activation map over the input image
        overlayed_img = cv2.addWeighted(input_img, alpha, cam, beta, gamma)
       
        # Displaying the overlaid class activation mapping
        visualize_gradcam(overlayed_img, subjects[i], 'turbo', 'grad-cam_blending')


# Revising the final model extracted features
gradcam_subject_analysis
